In [65]:
import glob
import codecs
import nltk
import re
from __future__ import absolute_import, division, print_function
import pprint
from nltk.corpus import stopwords

In [5]:
books_file_paths = sorted (glob.glob("./word_vectors_game_of_thrones-LIVE/data/*"))

In [121]:
def rawBook_to_processed(abook):
    rawbookstring = u''
    with codecs.open(abook, "r", "utf-8") as book_file:
        rawbookstring += book_file.read()
    tokenizer = nltk.data.load('/Users/Zhanik/nltk_data/tokenizers/punkt/english.pickle')
    
    
    raw_sentences = tokenizer.tokenize(rawbookstring)
    
    nltk_words = list(stopwords.words('english'))
    
    def sentence_to_wordlist(raw):
        clean = re.sub("[^a-zA-Z]"," ", raw)
        words = clean.split()
        lowercase = [word.strip().lower() for word in words]
        filtered_word_list = lowercase[:]  # make a copy of the word_list
        
        for word in lowercase:  # iterate over word_list
            if word in nltk_words: 
                filtered_word_list.remove(word) #
        return filtered_word_list
    
    words = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            words.append(sentence_to_wordlist(raw_sentence))
    return words


In [67]:
words1 = rawBook_to_processed(books_file_paths[0])

In [68]:
flat_list = [item for sublist in words1 for item in sublist]

In [122]:
books = []

In [123]:
for afile in books_file_paths:
    books.append ( [item.lower() for sublist in rawBook_to_processed(afile) for item in sublist] )

In [115]:
len(books)

5

In [30]:
from gensim import corpora

In [124]:
dictionary = corpora.Dictionary(books)
dictionary.save('./LDApickles/Got.dict')  # store the dictionary, for future reference
print(dictionary)

Dictionary(22805 unique tokens: [u'fawn', u'mustachio', u'woods', u'clotted', u'spiders']...)


In [73]:
print(dictionary.token2id)

{u'fawn': 0, u'mustachio': 17128, u'woods': 6, u'clotted': 7, u'spiders': 8, u'hanging': 1823, u'woody': 21955, u'sevens': 12953, u'lewyn': 12955, u'dragonbone': 18, u'scold': 19, u'clanked': 4530, u'caned': 23, u'unnecessarily': 22609, u'crossbar': 21970, u'lewys': 123, u'wracked': 17142, u'donnelwood': 13195, u'bringing': 36, u'wooded': 37, u'grueling': 12989, u'wooden': 39, u'crotch': 43, u'pisspot': 12714, u'festered': 46, u'guardsmen': 48, u'sooty': 50, u'pizzle': 20418, u'lefford': 53, u'scrapes': 17163, u'scraped': 63, u'snuggled': 65, u'errors': 17174, u'harsley': 13025, u'donyse': 13026, u'deferred': 19330, u'cooking': 69, u'pawed': 71, u'succumb': 13037, u'shocks': 13039, u'crouch': 13041, u'chins': 77, u'quagmires': 13049, u'chink': 85, u'natured': 86, u'kids': 19118, u'yronwoods': 20431, u'outdrink': 13062, u'climbed': 89, u'spotty': 13073, u'climber': 93, u'golden': 94, u'cobblers': 20435, u'kinslayers': 95, u'lengthen': 98, u'stern': 101, u'quentyn': 102, u'forbore': 2044

In [48]:
print(dictionary.doc2bow(books[0]))

[(0, 4), (1, 3), (2, 1), (3, 12), (4, 94), (5, 1), (6, 28), (7, 1), (8, 2), (9, 13), (10, 1), (11, 1), (12, 16), (13, 1), (14, 1), (15, 1), (16, 3), (17, 2), (18, 3), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1516), (26, 1), (27, 5), (28, 8), (29, 7), (30, 5), (31, 3), (32, 54), (33, 2), (34, 51), (35, 9), (36, 22), (37, 3), (38, 3), (39, 60), (40, 4), (41, 4), (42, 2), (43, 1), (44, 8), (45, 4), (46, 2), (47, 1), (48, 23), (49, 2), (50, 2), (51, 3), (52, 22), (53, 5), (54, 1), (55, 1), (56, 15), (57, 1), (58, 5), (59, 66), (60, 3), (61, 8), (62, 22), (63, 3), (64, 1), (65, 1), (66, 3), (67, 4), (68, 7), (69, 6), (70, 178), (71, 2), (72, 8), (73, 3), (74, 6), (75, 7), (76, 1), (77, 1), (78, 1), (79, 410), (80, 3), (81, 7), (82, 15), (83, 18), (84, 5), (85, 1), (86, 1), (87, 6), (88, 2), (89, 54), (90, 4), (91, 4), (92, 1), (93, 2), (94, 98), (95, 1), (96, 3), (97, 12), (98, 1), (99, 4), (100, 148), (101, 12), (102, 1), (103, 1), (104, 1), (105, 53), (106, 1), (107, 1),

In [117]:
corpus = [dictionary.doc2bow(text) for text in books]

In [51]:
import gensim.models.ldamodel as lda

In [118]:
mymodel = lda.LdaModel(corpus, id2word=dictionary, num_topics=100)

In [149]:
mymodel.print_topic(88)

u'0.010*"said" + 0.010*"lord" + 0.008*"ser" + 0.008*"would" + 0.008*"one" + 0.006*"could" + 0.006*"back" + 0.005*"well" + 0.005*"man" + 0.004*"king"'

In [151]:
mymodel.print_topic(10, topn=100)

u'0.008*"said" + 0.008*"lord" + 0.006*"would" + 0.006*"one" + 0.006*"men" + 0.005*"could" + 0.005*"ser" + 0.005*"man" + 0.004*"king" + 0.004*"like" + 0.004*"back" + 0.004*"old" + 0.004*"father" + 0.004*"jon" + 0.003*"see" + 0.003*"tyrion" + 0.003*"hand" + 0.003*"even" + 0.003*"know" + 0.003*"black" + 0.003*"come" + 0.003*"long" + 0.003*"time" + 0.002*"us" + 0.002*"told" + 0.002*"never" + 0.002*"red" + 0.002*"boy" + 0.002*"thought" + 0.002*"well" + 0.002*"head" + 0.002*"night" + 0.002*"must" + 0.002*"made" + 0.002*"face" + 0.002*"first" + 0.002*"eyes" + 0.002*"lady" + 0.002*"good" + 0.002*"take" + 0.002*"away" + 0.002*"white" + 0.002*"though" + 0.002*"go" + 0.002*"let" + 0.002*"way" + 0.002*"came" + 0.002*"little" + 0.002*"dead" + 0.002*"tell" + 0.002*"looked" + 0.002*"queen" + 0.002*"blood" + 0.002*"jaime" + 0.002*"still" + 0.002*"arya" + 0.002*"took" + 0.002*"asked" + 0.002*"great" + 0.002*"dany" + 0.002*"called" + 0.002*"bran" + 0.002*"robert" + 0.002*"much" + 0.002*"gold" + 0.002*"s